In [30]:
import pandas as pd

path = '/Users/brunobarbieri/Library/CloudStorage/OneDrive-UniversityofPisa/TA_Project/data/'
df = pd.read_csv(path + "gaia_bruno_lemmatized_labelled.csv")

In [31]:
import ast

df['lemmatized_stanzas'] = df['lemmatized_stanzas'].apply(ast.literal_eval)
df

,Unnamed: 0,id,title,artist,year,views,features,is_country,is_pop,is_rap,is_rb,is_rock,stanza_number,is_chorus,lemmatized_stanzas,label
0,0,0,Something in the Water,Pokey LaFarge,2015,10902,{''},True,False,False,False,False,0,False,"[she, get, a, broke, down, el, camino, in, the...",anger
1,1,0,Something in the Water,Pokey LaFarge,2015,10902,{''},True,False,False,False,False,1,True,"[something, in, the, water, something, in, the...",anger
2,2,0,Something in the Water,Pokey LaFarge,2015,10902,{''},True,False,False,False,False,2,False,"[she, do, her, makeup, and, hair, to, cook, fr...",anticipation
3,3,0,Something in the Water,Pokey LaFarge,2015,10902,{''},True,False,False,False,False,4,False,"[my, hoosi, girl, be, so, fine, shake, the, wa...",fear
4,4,0,Something in the Water,Pokey LaFarge,2015,10902,{''},True,False,False,False,False,5,True,"[something, in, the, water, something, in, the...",fear
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29205,29205,5554,That Kind of Song,The Nostalgia Critic,2013,19,"{'PawDugan', 'Elisa Hansen'}",False,True,False,False,False,12,False,"[elisa/, paw, what, if, we, be, in, that, kind...",disgust
29206,29206,5554,That Kind of Song,The Nostalgia Critic,2013,19,"{'PawDugan', 'Elisa Hansen'}",False,True,False,False,False,13,False,"[puppeteer, kermit, the, frog, voice, so, I, h...",disgust
29207,29207,5554,That Kind of Song,The Nostalgia Critic,2013,19,"{'PawDugan', 'Elisa Hansen'}",False,True,False,False,False,14,False,"[elisa/, paw, look, into, each, other, 's, eye...",disgust
29208,29208,5554,That Kind of Song,The Nostalgia Critic,2013,19,"{'PawDugan', 'Elisa Hansen'}",False,True,False,False,False,15,False,"[paw, and, you, would, be, the, intriguing, in...",joy


In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
# import numpy as np


# Step 1: Convert token lists back into space-separated strings
# (needed for vectorizer)
# texts_str = df['lemmatized_stanzas'].apply(
#     lambda tokens: " ".join(tokens)
# )
# print(texts_str)
df['text_str'] = df['lemmatized_stanzas'].apply(lambda x: ' '.join(x))


# Step 3: Encode using TfidfVectorizer (TF-IDF approach)
# tfidf_vectorizer = TfidfVectorizer()
# X_tfidf = tfidf_vectorizer.fit_transform(df['text_str'])
# print("\nTfidfVectorizer Encoding (TF-IDF):")
# X_tfidf.toarray()

# # Step 4: Example using Word Embeddings (GloVe-like) for each text entry
# # Simulating word embeddings (replace this with actual embeddings like GloVe or Word2Vec)
# # embedding_dim = 50  # Example embedding dimension size
# # def get_word_embeddings(tokens):
# #     return np.array([np.random.randn(embedding_dim) for token in tokens])

# # word_embeddings = df['lemmatized_stanzas'].apply(get_word_embeddings)

# # print("\nWord Embedding Encoding (Random GloVe-like vectors):")
# # print(word_embeddings)

In [33]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df['text_str'], df['label'], test_size=0.3, random_state=42
)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report


# Initialize and train the Random Forest
# rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
# rf_model.fit(X_train, y_train)

# Evaluate on the test set
# y_pred = rf_model.predict(X_test)
# print(classification_report(y_test, y_pred))

# Define the pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),               # TF-IDF encoding
    ('rf', RandomForestClassifier(random_state=42))  # Random Forest model
])

param_distributions = {
    'tfidf__max_features': [500, 1000, 5000, None],  # Maximum number of features for TF-IDF
    'tfidf__ngram_range': [(1, 1), (1, 2)],          # Unigrams or bigrams
    'rf__n_estimators': [50, 100, 200, 300],         # Number of trees
    'rf__max_depth': [None, 10, 20, 30],             # Tree depth
    'rf__min_samples_split': [2, 5, 10],             # Minimum samples per split
    'rf__min_samples_leaf': [1, 2, 4],               # Minimum samples per leaf
    'rf__bootstrap': [True, False],                  # Bootstrap sampling
}

# RandomizedSearchCV setup
random_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_distributions,
    n_iter=20,                                  # Number of random combinations to try
    cv=5,                                       # 5-fold cross-validation
    scoring='accuracy',                         # Metric to optimize
    verbose=2,
    random_state=42,
    n_jobs=-1                                   # Use all available cores
)


In [35]:
# Fit RandomizedSearchCV to the data
random_search.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] END rf__bootstrap=True, rf__max_depth=20, rf__min_samples_leaf=4, rf__min_samples_split=10, rf__n_estimators=300, tfidf__max_features=5000, tfidf__ngram_range=(1, 1); total time=  14.0s
[CV] END rf__bootstrap=True, rf__max_depth=20, rf__min_samples_leaf=4, rf__min_samples_split=10, rf__n_estimators=300, tfidf__max_features=5000, tfidf__ngram_range=(1, 1); total time=  14.0s
[CV] END rf__bootstrap=True, rf__max_depth=20, rf__min_samples_leaf=4, rf__min_samples_split=10, rf__n_estimators=300, tfidf__max_features=5000, tfidf__ngram_range=(1, 1); total time=  14.1s
[CV] END rf__bootstrap=True, rf__max_depth=20, rf__min_samples_leaf=4, rf__min_samples_split=10, rf__n_estimators=300, tfidf__max_features=5000, tfidf__ngram_range=(1, 1); total time=  14.2s
[CV] END rf__bootstrap=True, rf__max_depth=20, rf__min_samples_leaf=4, rf__min_samples_split=10, rf__n_estimators=300, tfidf__max_features=5000, tfidf__ngram_range=(1, 1); to

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                             ('rf',
                                              RandomForestClassifier(random_state=42))]),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'rf__bootstrap': [True, False],
                                        'rf__max_depth': [None, 10, 20, 30],
                                        'rf__min_samples_leaf': [1, 2, 4],
                                        'rf__min_samples_split': [2, 5, 10],
                                        'rf__n_estimators': [50, 100, 200, 300],
                                        'tfidf__max_features': [500, 1000, 5000,
                                                                None],
                                        'tfidf__ngram_range': [(1, 1), (1, 2)]},
                   random_state=42, scoring='accuracy', verbose=2)

In [ ]:
# Best parameters and cross-validation accuracy
print(f"Best Parameters: {random_search.best_params_}")
print(f"Best Cross-Validation Accuracy: {random_search.best_score_}")

# Predict on the test set
y_pred = random_search.best_estimator_.predict(X_test)
print(classification_report(y_test, y_pred))

Best Parameters: {'tfidf__ngram_range': (1, 1), 'tfidf__max_features': None, 'rf__n_estimators': 200, 'rf__min_samples_split': 10, 'rf__min_samples_leaf': 1, 'rf__max_depth': None, 'rf__bootstrap': False}
Best Cross-Validation Accuracy: 0.4046065746193646
              precision    recall  f1-score   support

       anger       0.40      0.94      0.56      2979
anticipation       0.57      0.10      0.18       921
     disgust       0.41      0.06      0.10       524
        fear       0.48      0.25      0.33      1667
         joy       0.37      0.11      0.17       445
     sadness       0.58      0.15      0.24      1110
    surprise       0.48      0.09      0.15       494
       trust       0.48      0.07      0.13       623

    accuracy                           0.42      8763
   macro avg       0.47      0.22      0.23      8763
weighted avg       0.47      0.42      0.33      8763



In [37]:
# saves the model
import joblib

model_path = '/Users/brunobarbieri/Library/CloudStorage/OneDrive-UniversityofPisa/TA_Project/models/'

import joblib
joblib.dump(
    random_search.best_estimator_,
    model_path + 'best_rf_pipeline.pkl'
)

['/Users/brunobarbieri/Library/CloudStorage/OneDrive-UniversityofPisa/TA_Project/models/best_rf_pipeline.pkl']

# SVM

In [ ]:
from sklearn.svm import SVC

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),               # TF-IDF encoding
    ('rf', SVC(random_state=42))  # Random Forest model
])

param_distributions = {
    'tfidf__max_features': [500, 1000, 5000, None],    # Maximum features for TF-IDF
    'tfidf__ngram_range': [(1, 1), (1, 2)],            # Unigrams or bigrams
    'svm__C': [0.1, 1, 10, 100],                       # Regularization strength
    'svm__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],  # SVM kernel types
    'svm__gamma': ['scale', 'auto'],                   # Kernel coefficient
}


# RandomizedSearchCV setup
random_search_svm = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_distributions,
    n_iter=20,                                  # Number of random combinations to try
    cv=5,                                       # 5-fold cross-validation
    scoring='accuracy',                         # Metric to optimize
    verbose=2,
    random_state=42,
    n_jobs=-1                                   # Use all available cores
)

In [ ]:
# Fit RandomizedSearchCV to the data
random_search_svm.fit(X_train, y_train)

In [ ]:
# Best parameters and cross-validation accuracy
print(f"Best Parameters: {random_search.best_params_}")
print(f"Best Cross-Validation Accuracy: {random_search.best_score_}")

# Predict on the test set
y_pred = random_search.best_estimator_.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
# saves the model
import joblib

model_path = '/Users/brunobarbieri/Library/CloudStorage/OneDrive-UniversityofPisa/TA_Project/models/'

import joblib
joblib.dump(
    random_search.best_estimator_,
    model_path + 'best_svm_pipeline.pkl'
)